### 1. 함수 매핑
- 함수 매핑 : 시리즈 또는 데이터프레임의 개별 원소를 특정 함수에 일대일 대응 시키는 과정
- 사용자가 직접 만든 함수 적용 가능

#### 1-1. 개별 원소에 함수 매핑

##### 시리즈 원소에 함수 매핑
- apply() 메소드 적용
    - 인자로 전달하는 매핑 함수에 모든 원소 하나씩 입력하고 함수의 리턴값 돌려받음.
    - 시리즈 원소 개수만큼 리턴값 받아서 같은 크기의 시리즈 객체로 반환

In [1]:
import seaborn as sns

titanic = sns.load_dataset('titanic')
df = titanic.loc[:,['age','fare']]
df['ten'] = 10
print(df.head())

    age     fare  ten
0  22.0   7.2500   10
1  38.0  71.2833   10
2  26.0   7.9250   10
3  35.0  53.1000   10
4  35.0   8.0500   10


In [2]:
# 사용자 함수 정의
def add_10(n):
    return n + 10

def add_two_obj(a,b):
    return a + b

print(add_10(10))
print(add_two_obj(10,10))

20
20


In [3]:
# 시리즈 객체에 적용
sr1 = df['age'].apply(add_10)
print(sr1.head())
print("\n")

sr2 = df['age'].apply(add_two_obj, b = 10)
print(sr2.head())
print("\n")

sr3 = df['age'].apply(lambda x: add_10(x))
print(sr3.head())

0    32.0
1    48.0
2    36.0
3    45.0
4    45.0
Name: age, dtype: float64


0    32.0
1    48.0
2    36.0
3    45.0
4    45.0
Name: age, dtype: float64


0    32.0
1    48.0
2    36.0
3    45.0
4    45.0
Name: age, dtype: float64


##### 데이터프레임 원소에 함수 매핑
- applymap()
    - 원소 원래 위치에 매핑 함수 리턴값 입력하여 동일한 형태의 데이터프레임 만들어짐

In [4]:
df_map = df.applymap(add_10)
df_map.head()

,age,fare,ten
0,32.0,17.2500,20
1,48.0,81.2833,20
2,36.0,17.9250,20
3,45.0,63.1000,20
4,45.0,18.0500,20


#### 1-2. 시리즈 객체에 함수 매핑

##### 데이터프레임 각 열에 함수 매핑
- apply(axis=0) : 모든 열 하나씩 분리하여 매핑 함수 인자로 각 열(시리즈)이 전달됨.

In [7]:
def missing_value(series):
    return series.isnull() # 시리즈 반환

result = df.apply(missing_value, axis=0)
print(result.head())
print(type(result)) 

     age   fare    ten
0  False  False  False
1  False  False  False
2  False  False  False
3  False  False  False
4  False  False  False
<class 'pandas.core.frame.DataFrame'>


In [8]:
def min_max(x):
    return x.max() - x.min() # 값 반환

result = df.apply(min_max) # 기본값 axis = 0
print(result)
print(type(result))

age      79.5800
fare    512.3292
ten       0.0000
dtype: float64
<class 'pandas.core.series.Series'>


##### 데이터프레임 각 행에 함수 매핑
- apply(axis=1)

In [9]:
df['add'] = df.apply(lambda x: add_two_obj(x['age'],x['ten']), axis=1) # 행별 연산
df.head()

,age,fare,ten,add
0,22.0,7.2500,10,32.0
1,38.0,71.2833,10,48.0
2,26.0,7.9250,10,36.0
3,35.0,53.1000,10,45.0
4,35.0,8.0500,10,45.0


#### 1-3. 데이터프레임 객체에 함수 매핑
- pipe() 메소드 활용

In [10]:
# 각 열의 NaN 찾기
def missing_value(x):
    return x.isnull()
# 각 열의 NaN 개수 반환
def missing_count(x):
    return missing_value(x).sum()
# 데이터프레임의 총 NaN 개수
def total_number_missing(x):
    return missing_count(x).sum()

In [11]:
result_df = df.pipe(missing_value)
print(result_df.head())
print(type(result_df))

     age   fare    ten    add
0  False  False  False  False
1  False  False  False  False
2  False  False  False  False
3  False  False  False  False
4  False  False  False  False
<class 'pandas.core.frame.DataFrame'>


In [13]:
result_series = df.pipe(missing_count)
print(result_series)
print(type(result_series))

age     177
fare      0
ten       0
add     177
dtype: int64
<class 'pandas.core.series.Series'>


In [15]:
result_value = df.pipe(total_number_missing)
print(result_value)
print(type(result_value))

354
<class 'numpy.int64'>


### 2. 열 재구성

#### 2-1. 열 순서 변경
- 열 이름을 원하는 순서대로 정리해서 리스트 만든 후, 데이터프레임에서 열 다시 선택

In [21]:
df = titanic.loc[0:4, 'survived':'age']
print(df,'\n')

columns = list(df.columns.values)
print(columns, "\n")

# 열 이름 알파벳 순으로 정렬
columns_sorted = sorted(columns)
df_sorted = df[columns_sorted]
print(df_sorted,"\n")

# 기준 순서의 정반대 역순으로 정렬
columns_reversed = list(reversed(columns))
df_reversed = df[columns_reversed]
print(df_reversed, "\n")

# 임의의 순서로 열 이름 재배치
columns_customed = ['pclass','sex','age','survived']
df_customed = df[columns_customed]
print(df_customed)

   survived  pclass     sex   age
0         0       3    male  22.0
1         1       1  female  38.0
2         1       3  female  26.0
3         1       1  female  35.0
4         0       3    male  35.0 

['survived', 'pclass', 'sex', 'age'] 

    age  pclass     sex  survived
0  22.0       3    male         0
1  38.0       1  female         1
2  26.0       3  female         1
3  35.0       1  female         1
4  35.0       3    male         0 

    age     sex  pclass  survived
0  22.0    male       3         0
1  38.0  female       1         1
2  26.0  female       3         1
3  35.0  female       1         1
4  35.0    male       3         0 

   pclass     sex   age  survived
0       3    male  22.0         0
1       1  female  38.0         1
2       3  female  26.0         1
3       1  female  35.0         1
4       3    male  35.0         0


#### 2-2. 열 분리
- 연월일 -> 연,월,일 로 구분
- 이름 -> 성 ,이름으로 구분

In [23]:
import pandas as pd

df = pd.read_excel('./data/주가데이터.xlsx')
print(df.head(),"\n")
print(df.dtypes, "\n")

         연월일   당일종가  전일종가     시가     고가     저가     거래량
0 2018-07-02  10100   600  10850  10900  10000  137977
1 2018-06-29  10700   300  10550  10900   9990  170253
2 2018-06-28  10400   500  10900  10950  10150  155769
3 2018-06-27  10900   100  10800  11050  10500  133548
4 2018-06-26  10800   350  10900  11000  10700   63039 

연월일     datetime64[ns]
당일종가             int64
전일종가             int64
시가               int64
고가               int64
저가               int64
거래량              int64
dtype: object 



In [25]:
# 문자열 메소드 사용을 위해 자료형 변경
df['연월일'] = df['연월일'].astype('str')
dates = df['연월일'].str.split('-')
print(dates.head(),"\n")

# 분리된 정보 각각 새로운 열에 담아 df에 추가하기
df['연'] = dates.str.get(0)
df['월'] = dates.str.get(1)
df['일'] = dates.str.get(2)
print(df.head())

0    [2018, 07, 02]
1    [2018, 06, 29]
2    [2018, 06, 28]
3    [2018, 06, 27]
4    [2018, 06, 26]
Name: 연월일, dtype: object 

          연월일   당일종가  전일종가     시가     고가     저가     거래량     연   월   일
0  2018-07-02  10100   600  10850  10900  10000  137977  2018  07  02
1  2018-06-29  10700   300  10550  10900   9990  170253  2018  06  29
2  2018-06-28  10400   500  10900  10950  10150  155769  2018  06  28
3  2018-06-27  10900   100  10800  11050  10500  133548  2018  06  27
4  2018-06-26  10800   350  10900  11000  10700   63039  2018  06  26


### 3. 필터링
- 특정 조건식을 만족하는 원소만 따로 추출

#### 3-1. 불린 인덱싱

In [26]:
# 나이가 10대인 승객만 따로 선택
mask1 = (titanic.age >=10) & (titanic.age < 20)
df_teenage = titanic.loc[mask1, :]
print(df_teenage.head())

    survived  pclass     sex   age  sibsp  parch      fare embarked   class  \
9          1       2  female  14.0      1      0   30.0708        C  Second   
14         0       3  female  14.0      0      0    7.8542        S   Third   
22         1       3  female  15.0      0      0    8.0292        Q   Third   
27         0       1    male  19.0      3      2  263.0000        S   First   
38         0       3  female  18.0      2      0   18.0000        S   Third   

      who  adult_male deck  embark_town alive  alone  
9   child       False  NaN    Cherbourg   yes  False  
14  child       False  NaN  Southampton    no   True  
22  child       False  NaN   Queenstown   yes   True  
27    man        True    C  Southampton    no  False  
38  woman       False  NaN  Southampton    no  False  


In [28]:
# 나이가 10세 미만이고 여성인 승객만 따로 선택
mask2 = (titanic.age < 10) & (titanic.sex == 'female')
df_female_under10 = titanic.loc[mask2,:]
df_female_under10.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
10,1,3,female,4.0,1,1,16.7000,S,Third,child,False,G,Southampton,yes,False
24,0,3,female,8.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
43,1,2,female,3.0,1,2,41.5792,C,Second,child,False,NaN,Cherbourg,yes,False
58,1,2,female,5.0,1,2,27.7500,S,Second,child,False,NaN,Southampton,yes,False
119,0,3,female,2.0,4,2,31.2750,S,Third,child,False,NaN,Southampton,no,False


In [30]:
# 나이가 10세 미만 또는 60세 이상인 승객의 age, sex, alone 열만 선택
mask3 = (titanic.age < 10) | (titanic.age >= 60)
df_under10_morethan60 = titanic.loc[mask3,['age','sex','alone']]
df_under10_morethan60.head()

,age,sex,alone
7,2.0,male,False
10,4.0,female,False
16,2.0,male,False
24,8.0,female,False
33,66.0,male,True


#### 3-2. isin() 메소드 활용

In [31]:
# 출력할 최대 열의 개수 변경
pd.set_option('display.max_columns',10)

# 함께 탑승한 형제 또는 배우자 수가 3,4,5인 승객만 따로 추출
mask3 = titanic['sibsp'] == 3
mask4 = titanic['sibsp'] == 4
mask5 = titanic['sibsp'] == 5
df_boolean = titanic[mask3|mask4|mask5]
df_boolean.head()

,survived,pclass,sex,age,sibsp,...,adult_male,deck,embark_town,alive,alone
7,0,3,male,2.0,3,...,False,NaN,Southampton,no,False
16,0,3,male,2.0,4,...,False,NaN,Queenstown,no,False
24,0,3,female,8.0,3,...,False,NaN,Southampton,no,False
27,0,1,male,19.0,3,...,True,C,Southampton,no,False
50,0,3,male,7.0,4,...,False,NaN,Southampton,no,False


In [32]:
# isin() 메소드 활용
isin_filter = titanic['sibsp'].isin([3,4,5])
df_isin = titanic[isin_filter]
df_isin.head()

,survived,pclass,sex,age,sibsp,...,adult_male,deck,embark_town,alive,alone
7,0,3,male,2.0,3,...,False,NaN,Southampton,no,False
16,0,3,male,2.0,4,...,False,NaN,Queenstown,no,False
24,0,3,female,8.0,3,...,False,NaN,Southampton,no,False
27,0,1,male,19.0,3,...,True,C,Southampton,no,False
50,0,3,male,7.0,4,...,False,NaN,Southampton,no,False


### 4. 데이터프레임 합치기

#### 4-1. 데이터프레임 연결
- concat() : 기존 데이터프레임 형태 유지하면서 이어붙이는 개념
    - default
        - axis = 0 / 위 아래 방향으로 연결됨 
        - 열 이름 join='outer'이 기본 , 열이름 합집합 구성
        - join ='inner'의 경우 교집합이 기준

In [37]:
df1 = pd.DataFrame({'a':['a0','a1','a2','a3'],
                    'b':['b0','b1','b2','b3'],
                    'c':['c0','c1','c2','c3']}
                    ,index = [0,1,2,3])
df2 = pd.DataFrame({'a':['a2','a3','a4','a5'],
                    'b':['b2','b3','b4','b5'],
                    'c':['c2','c3','c4','c5'],
                    'd':['d2','d3','d4','d5']}
                   ,index = [2,3,4,5])
result1 = pd.concat([df1,df2])
result1

,a,b,c,d
0,a0,b0,c0,NaN
1,a1,b1,c1,NaN
2,a2,b2,c2,NaN
3,a3,b3,c3,NaN
2,a2,b2,c2,d2
3,a3,b3,c3,d3
4,a4,b4,c4,d4
5,a5,b5,c5,d5


In [36]:
# ignore_index = True : 기존 행 인덱스 무시하고 새로운 행 인덱스 생성
# 정수형 인덱스가 새롭게 설정됨
result2 = pd.concat([df1,df2], ignore_index=True)
result2

,a,b,c,d
0,a0,b0,c0,NaN
1,a1,b1,c1,NaN
2,a2,b2,c2,NaN
3,a3,b3,c3,NaN
4,a2,b2,c2,d2
5,a3,b3,c3,d3
6,a4,b4,c4,d4
7,a5,b5,c5,d5


In [38]:
# 좌우 방향으로 연결하기
result3 = pd.concat([df1,df2], axis=1)
result3

,a,b,c,a,b,c,d
0,a0,b0,c0,NaN,NaN,NaN,NaN
1,a1,b1,c1,NaN,NaN,NaN,NaN
2,a2,b2,c2,a2,b2,c2,d2
3,a3,b3,c3,a3,b3,c3,d3
4,NaN,NaN,NaN,a4,b4,c4,d4
5,NaN,NaN,NaN,a5,b5,c5,d5


In [39]:
# join = 'inner' 옵션 적용
result3_in = pd.concat([df1,df2], axis=1, join='inner')
result3_in

,a,b,c,a,b,c,d
2,a2,b2,c2,a2,b2,c2,d2
3,a3,b3,c3,a3,b3,c3,d3


In [41]:
# 시리즈 만들기
sr1 = pd.Series(['e0','e1','e2','e3'], name='e')
sr2 = pd.Series(['f0','f1','f2'], name = 'f', index=[3,4,5])
sr3 = pd.Series(['g0','g1','g2','g3'], name='g')
# 데이터프레임 + 시리즈
result4 = pd.concat([df1,sr1], axis=1)
print(result4,"\n")

result5 = pd.concat([df2,sr2], axis=1, sort=True)
print(result5,"\n")


    a   b   c   e
0  a0  b0  c0  e0
1  a1  b1  c1  e1
2  a2  b2  c2  e2
3  a3  b3  c3  e3 

    a   b   c   d    f
2  a2  b2  c2  d2  NaN
3  a3  b3  c3  d3   f0
4  a4  b4  c4  d4   f1
5  a5  b5  c5  d5   f2 



In [42]:
# 시리즈끼리 연결
result6 = pd.concat([sr1,sr3], axis=1)
print(result6,"\n")

result7 = pd.concat([sr1,sr3], axis=0)
print(result7,"\n")

    e   g
0  e0  g0
1  e1  g1
2  e2  g2
3  e3  g3 

0    e0
1    e1
2    e2
3    e3
0    g0
1    g1
2    g2
3    g3
dtype: object 



#### 4-2. 데이터프레임 병합
- merge()
    - SQL의 join 명령과 비슷한 방식으로 어떤 기준으로 두 데이터프레임 병합
    - 키 : 기준이 되는 열이나 인덱스 / 반드시 양쪽 데이터프레임에 모두 존재해야함.

In [43]:
pd.set_option('display.max_columns',10) # 출력할 최대 열 개수
pd.set_option('display.max_colwidth',20) # 출력할 열 너비
pd.set_option('display.unicode.east_asian_width',True) # 유니코드 사용 너비 조정 

# 주식데이터
df1 = pd.read_excel('./data/stock price.xlsx')
df2 = pd.read_excel('./data/stock valuation.xlsx')

print(df1.head())
print(df2.head())

       id    stock_name          value   price
0  128940      한미약품   59385.666667  421000
1  130960        CJ E&M   58540.666667   98900
2  138250    엔에스쇼핑   14558.666667   13200
3  139480        이마트  239230.833333  254500
4  142280  녹십자엠에스     468.833333   10200
       id            name           eps     bps        per       pbr
0  130960          CJ E&M   6301.333333   54068  15.695091  1.829178
1  136480            하림    274.166667    3551  11.489362  0.887074
2  138040  메리츠금융지주   2122.333333   14894   6.313806  0.899691
3  139480          이마트  18268.166667  295780  13.931338  0.860437
4  145990          삼양사   5741.000000  108090  14.283226  0.758627


In [44]:
# 데이터 합치기 - 교집합
merge_inner = pd.merge(df1,df2)
merge_inner

,id,stock_name,value,price,name,eps,bps,per,pbr
0,130960,CJ E&M,58540.666667,98900,CJ E&M,6301.333333,54068,15.695091,1.829178
1,139480,이마트,239230.833333,254500,이마트,18268.166667,295780,13.931338,0.860437
2,145990,삼양사,82750.000000,82000,삼양사,5741.000000,108090,14.283226,0.758627
3,185750,종근당,40293.666667,100500,종근당,3990.333333,40684,25.185866,2.470259
4,204210,모두투어리츠,3093.333333,3475,모두투어리츠,85.166667,5335,40.802348,0.651359


In [46]:
# 데이터프레임 합치기 - 합집합
merge_outer = pd.merge(df1,df2, how='outer', on='id')
merge_outer.head()

,id,stock_name,value,price,name,eps,bps,per,pbr
0,128940,한미약품,59385.666667,421000.0,NaN,NaN,NaN,NaN,NaN
1,130960,CJ E&M,58540.666667,98900.0,CJ E&M,6301.333333,54068.0,15.695091,1.829178
2,138250,엔에스쇼핑,14558.666667,13200.0,NaN,NaN,NaN,NaN,NaN
3,139480,이마트,239230.833333,254500.0,이마트,18268.166667,295780.0,13.931338,0.860437
4,142280,녹십자엠에스,468.833333,10200.0,NaN,NaN,NaN,NaN,NaN


In [47]:
# 데이터프레임 합치기 - 왼쪽 데이터프레임 기준, 키 값 분리
# 좌우 데이터프레임에 각각 다르게 키 지정 가능
merge_left = pd.merge(df1, df2, how='left', left_on='stock_name', right_on='name')
merge_left

,id_x,stock_name,value,price,id_y,name,eps,bps,per,pbr
0,128940,한미약품,59385.666667,421000,NaN,NaN,NaN,NaN,NaN,NaN
1,130960,CJ E&M,58540.666667,98900,130960.0,CJ E&M,6301.333333,54068.0,15.695091,1.829178
2,138250,엔에스쇼핑,14558.666667,13200,NaN,NaN,NaN,NaN,NaN,NaN
3,139480,이마트,239230.833333,254500,139480.0,이마트,18268.166667,295780.0,13.931338,0.860437
4,142280,녹십자엠에스,468.833333,10200,NaN,NaN,NaN,NaN,NaN,NaN
5,145990,삼양사,82750.000000,82000,145990.0,삼양사,5741.000000,108090.0,14.283226,0.758627
6,185750,종근당,40293.666667,100500,185750.0,종근당,3990.333333,40684.0,25.185866,2.470259
7,192400,쿠쿠홀딩스,179204.666667,177500,NaN,NaN,NaN,NaN,NaN,NaN
8,199800,툴젠,-2514.333333,115400,NaN,NaN,NaN,NaN,NaN,NaN
9,204210,모두투어리츠,3093.333333,3475,204210.0,모두투어리츠,85.166667,5335.0,40.802348,0.651359


In [48]:
# 데이터프레임 합치기 - 오른쪽 데이터프레임 기준, 키 값 분리
merge_right = pd.merge(df1,df2, how='right', left_on='stock_name',right_on='name')
merge_right

,id_x,stock_name,value,price,id_y,name,eps,bps,per,pbr
0,130960.0,CJ E&M,58540.666667,98900.0,130960,CJ E&M,6301.333333,54068,15.695091,1.829178
1,NaN,NaN,NaN,NaN,136480,하림,274.166667,3551,11.489362,0.887074
2,NaN,NaN,NaN,NaN,138040,메리츠금융지주,2122.333333,14894,6.313806,0.899691
3,139480.0,이마트,239230.833333,254500.0,139480,이마트,18268.166667,295780,13.931338,0.860437
4,145990.0,삼양사,82750.000000,82000.0,145990,삼양사,5741.000000,108090,14.283226,0.758627
5,NaN,NaN,NaN,NaN,161390,한국타이어,5648.500000,51341,7.453306,0.820007
6,NaN,NaN,NaN,NaN,181710,NHN엔터테인먼트,2110.166667,78434,30.755864,0.827447
7,185750.0,종근당,40293.666667,100500.0,185750,종근당,3990.333333,40684,25.185866,2.470259
8,204210.0,모두투어리츠,3093.333333,3475.0,204210,모두투어리츠,85.166667,5335,40.802348,0.651359
9,NaN,NaN,NaN,NaN,207940,삼성바이오로직스,4644.166667,60099,89.790059,6.938551


In [50]:
# 불린 인덱싱과 결합하여 원하는 데이터 찾기
price = df1[df1['price'] < 50000]
print(price.head())
print("\n")
# df2에 밸류에이션 데이터 가진 회사 확인
value = pd.merge(price, df2)
print(value)

       id    stock_name         value  price
2  138250    엔에스쇼핑  14558.666667  13200
4  142280  녹십자엠에스    468.833333  10200
9  204210  모두투어리츠   3093.333333   3475


       id    stock_name        value  price          name        eps   bps  \
0  204210  모두투어리츠  3093.333333   3475  모두투어리츠  85.166667  5335   

         per       pbr  
0  40.802348  0.651359  


#### 4-3. 데이터프레임 결합
- join() : merge함수 기반으로 제작 
    - 두 데이터프레임의 행 인덱스 기준으로 결합
    - on = keys 옵션 설정 시 다른 열 기준으로 결합 가능

In [52]:
df1 = pd.read_excel('./data/stock price.xlsx', index_col='id')
df2 = pd.read_excel('./data/stock valuation.xlsx', index_col='id')
df3 = df1.join(df2) # how='left' 기본 적용
print(df3)

          stock_name          value   price          name           eps  \
id                                                                        
128940      한미약품   59385.666667  421000           NaN           NaN   
130960        CJ E&M   58540.666667   98900        CJ E&M   6301.333333   
138250    엔에스쇼핑   14558.666667   13200           NaN           NaN   
139480        이마트  239230.833333  254500        이마트  18268.166667   
142280  녹십자엠에스     468.833333   10200           NaN           NaN   
145990        삼양사   82750.000000   82000        삼양사   5741.000000   
185750        종근당   40293.666667  100500        종근당   3990.333333   
192400    쿠쿠홀딩스  179204.666667  177500           NaN           NaN   
199800          툴젠   -2514.333333  115400           NaN           NaN   
204210  모두투어리츠    3093.333333    3475  모두투어리츠     85.166667   

             bps        per       pbr  
id                                     
128940       NaN        NaN       NaN  
130960   54068.0  15.695091  1.

In [53]:
# 교집합
df4 = df1.join(df2, how='inner')
df4

,stock_name,value,price,name,eps,bps,per,pbr
id,,,,,,,,
130960,CJ E&M,58540.666667,98900,CJ E&M,6301.333333,54068,15.695091,1.829178
139480,이마트,239230.833333,254500,이마트,18268.166667,295780,13.931338,0.860437
145990,삼양사,82750.000000,82000,삼양사,5741.000000,108090,14.283226,0.758627
185750,종근당,40293.666667,100500,종근당,3990.333333,40684,25.185866,2.470259
204210,모두투어리츠,3093.333333,3475,모두투어리츠,85.166667,5335,40.802348,0.651359


### 5. 그룹 연산
- 데이터 집계, 변환, 필터링에 효율적
- 3단계 과정으로 이루어짐.
    1. 분할(split) : 데이터 특정 조건에 의해 분할 
    2. 적용(apply) : 데이터 집계, 변환, 필터링하는데 필요한 메소드 적용
    3. 결합(combine) : 2단계 처리 결과 하나로 결합

#### 5-1. 그룹 객체 만들기 (분할 단계)

In [56]:
# 1개 열 기준으로 그룹화
df = titanic.loc[:,['age','sex','class','fare','survived']]

print("승객 수:", len(df))
print(df.head())
print("\n")

# class 기준으로 분할
grouped = df.groupby(['class'])
print(grouped)

for key, group in grouped:
    print('* key :', key)
    print('* number :', len(group))
    print(group.head())

승객 수: 891
    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
1  38.0  female  First  71.2833         1
2  26.0  female  Third   7.9250         1
3  35.0  female  First  53.1000         1
4  35.0    male  Third   8.0500         0


* key : First
* number : 216
     age     sex  class     fare  survived
1   38.0  female  First  71.2833         1
3   35.0  female  First  53.1000         1
6   54.0    male  First  51.8625         0
11  58.0  female  First  26.5500         1
23  28.0    male  First  35.5000         1
* key : Second
* number : 184
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
17   NaN    male  Second  13.0000         1
20  35.0    male  Second  26.0000         0
21  34.0    male  Second  13.0000         1
* key : Third
* number : 491
    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1


In [59]:
# 연산 메소드 적용
average = grouped.mean()
print(average)

# 개별 그룹 선택하기
group3 = grouped.get_group('Third')
print(group3.head())

              age       fare  survived
class                                 
First   38.233441  84.154687  0.629630
Second  29.877630  20.662183  0.472826
Third   25.140620  13.675550  0.242363
    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1
4  35.0    male  Third   8.0500         0
5   NaN    male  Third   8.4583         0
7   2.0    male  Third  21.0750         0


In [60]:
# 여러 열을 기준으로 그룹화
grouped_two = df.groupby(['class','sex'])

for key, group in grouped_two:
    print('* key :', key)
    print('* number :', len(group))
    print(group.head())

* key : ('First', 'female')
* number : 94
     age     sex  class      fare  survived
1   38.0  female  First   71.2833         1
3   35.0  female  First   53.1000         1
11  58.0  female  First   26.5500         1
31   NaN  female  First  146.5208         1
52  49.0  female  First   76.7292         1
* key : ('First', 'male')
* number : 122
     age   sex  class      fare  survived
6   54.0  male  First   51.8625         0
23  28.0  male  First   35.5000         1
27  19.0  male  First  263.0000         0
30  40.0  male  First   27.7208         0
34  28.0  male  First   82.1708         0
* key : ('Second', 'female')
* number : 76
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
41  27.0  female  Second  21.0000         0
43   3.0  female  Second  41.5792         1
53  29.0  female  Second  26.0000         1
* key : ('Second', 'male')
* number : 108
     age   sex   class  fare  survived
17   NaN  mal

In [61]:
# 연산 메소드 적용
average_two = grouped_two.mean()
print(average_two)
print("\n")
print(type(average_two))

                     age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447


<class 'pandas.core.frame.DataFrame'>


In [62]:
# 개별 그룹 선택하기
group3f = grouped_two.get_group(('Third','female'))
print(group3f.head())

     age     sex  class     fare  survived
2   26.0  female  Third   7.9250         1
8   27.0  female  Third  11.1333         1
10   4.0  female  Third  16.7000         1
14  14.0  female  Third   7.8542         0
18  31.0  female  Third  18.0000         0


#### 5-2. 그룹 연산 메소드 (적용-결합 단계)

In [64]:
# 각 그룹에 대한 모든 열이 표준편차 집계하여 데이터프레임 반환
std_all = grouped.std()
print(std_all)
print("\n")
print(type(std_all))
print("\n")

# 각 그룹에 대한 fare 열의 표준편차 집계하여 시리즈로 반환
std_fare = grouped.fare.std()
print(std_fare)
print("\n")
print(type(std_fare))

              age       fare  survived
class                                 
First   14.802856  78.380373  0.484026
Second  14.001077  13.417399  0.500623
Third   12.495398  11.778142  0.428949


<class 'pandas.core.frame.DataFrame'>


class
First     78.380373
Second    13.417399
Third     11.778142
Name: fare, dtype: float64


<class 'pandas.core.series.Series'>


In [65]:
# 그룹 객체에 agg() 메소드 적용 - 사용자 정의 함수 인자로 전달
def min_max(x):
    return x.max() - x.min()
# 각 그룹의 최대값, 최소값 차이 계산하여 그룹별로 집계
agg_minmax = grouped.agg(min_max)
print(agg_minmax.head())

          age      fare  survived
class                            
First   79.08  512.3292         1
Second  69.33   73.5000         1
Third   73.58   69.5500         1


In [66]:
# 여러 함수를 각 열에 동일하게 적용하여 집계
agg_all = grouped.agg(['min','max'])
print(agg_all.head())
print("\n")

# 각 열마다 다른 함수 적용하여 집계
agg_sep = grouped.agg({'fare':['min','max'], 'age':'mean'})
print(agg_sep.head())

         age           sex       fare           survived    
         min   max     min   max  min       max      min max
class                                                       
First   0.92  80.0  female  male  0.0  512.3292        0   1
Second  0.67  70.0  female  male  0.0   73.5000        0   1
Third   0.42  74.0  female  male  0.0   69.5500        0   1


       fare                  age
        min       max       mean
class                           
First   0.0  512.3292  38.233441
Second  0.0   73.5000  29.877630
Third   0.0   69.5500  25.140620


In [68]:
# 그룹 연산 데이터 변환
# transform() : 그룹별로 구분하여 각 원소에 함수 적용하지만, 그룹별 집계 대신
# 각 원소의 본래 행 인덱스와 열이름 기준으로 연산 결과 반환

# 그룹별 age 열의 평균 집계 연산
age_mean = grouped.age.mean()
print(age_mean)
print("\n")
# 그룹별 age열의 표준편차 집계 연산
age_std = grouped.age.std()
print(age_std)
print("\n")

# 그룹 객체의 age 열을 iteration으로 z-score 계산하여 출력
for key, group in grouped.age:
    group_zscore = (group - age_mean.loc[key])/age_std.loc[key]
    print('* origin:',key)
    print(group_zscore.head(3))
    print("\n")

class
First     38.233441
Second    29.877630
Third     25.140620
Name: age, dtype: float64


class
First     14.802856
Second    14.001077
Third     12.495398
Name: age, dtype: float64


* origin: First
1   -0.015770
3   -0.218434
6    1.065103
Name: age, dtype: float64


* origin: Second
9    -1.134029
15    1.794317
17         NaN
Name: age, dtype: float64


* origin: Third
0   -0.251342
2    0.068776
4    0.789041
Name: age, dtype: float64




In [70]:
# z-score 계산하는 사용자 함수 정의
def z_score(x):
    return (x-x.mean())/x.std()

age_zscore = grouped.age.transform(z_score)
print(age_zscore.loc[[1,9,0]]) # 1,2,3 그룹의 첫 데이터 확인
print("\n")
print(len(age_zscore)) # 반환 값 길이
print("\n")
print(age_zscore.loc[0:9]) # 반환 값 출력
print("\n")
print(type(age_zscore)) # 반환 객체의 자료형

1   -0.015770
9   -1.134029
0   -0.251342
Name: age, dtype: float64


891


0   -0.251342
1   -0.015770
2    0.068776
3   -0.218434
4    0.789041
5         NaN
6    1.065103
7   -1.851931
8    0.148805
9   -1.134029
Name: age, dtype: float64


<class 'pandas.core.series.Series'>


In [71]:
# 그룹 객체필터링
# 데이터 개수가 200개 이상인 그룹만 필터링하여 데이터프레임으로 반환
grouped_filter = grouped.filter(lambda x: len(x) >= 200)
print(grouped_filter.head())
print("\n")
print(type(grouped_filter))

    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
1  38.0  female  First  71.2833         1
2  26.0  female  Third   7.9250         1
3  35.0  female  First  53.1000         1
4  35.0    male  Third   8.0500         0


<class 'pandas.core.frame.DataFrame'>


In [72]:
# age 열의 평균값이 30보다 작은 그룹만 선택
age_filter = grouped.filter(lambda x: x.age.mean() < 30)
print(age_filter.tail())
print("\n")
print(type(age_filter))

      age     sex   class    fare  survived
884  25.0    male   Third   7.050         0
885  39.0  female   Third  29.125         0
886  27.0    male  Second  13.000         0
888   NaN  female   Third  23.450         0
890  32.0    male   Third   7.750         0


<class 'pandas.core.frame.DataFrame'>


In [73]:
# 그룹 객체에 함수 매핑
agg_grouped = grouped.apply(lambda x: x.describe())
print(agg_grouped)

                     age        fare    survived
class                                           
First  count  186.000000  216.000000  216.000000
       mean    38.233441   84.154687    0.629630
       std     14.802856   78.380373    0.484026
       min      0.920000    0.000000    0.000000
       25%     27.000000   30.923950    0.000000
       50%     37.000000   60.287500    1.000000
       75%     49.000000   93.500000    1.000000
       max     80.000000  512.329200    1.000000
Second count  173.000000  184.000000  184.000000
       mean    29.877630   20.662183    0.472826
       std     14.001077   13.417399    0.500623
       min      0.670000    0.000000    0.000000
       25%     23.000000   13.000000    0.000000
       50%     29.000000   14.250000    0.000000
       75%     36.000000   26.000000    1.000000
       max     70.000000   73.500000    1.000000
Third  count  355.000000  491.000000  491.000000
       mean    25.140620   13.675550    0.242363
       std     12.49

In [74]:
age_zscore = grouped.age.apply(z_score)
age_zscore.head()

0   -0.251342
1   -0.015770
2    0.068776
3   -0.218434
4    0.789041
Name: age, dtype: float64

In [75]:
# age 열의 데이터 평균이 30보다 작은 그룹만 필터링해 출력
age_filter = grouped.apply(lambda x: x.age.mean() < 30)
print(age_filter)
print("\n")
for x in age_filter.index:
    if age_filter[x] == True:
        age_filter_df = grouped.get_group(x)
        print(age_filter_df.head())
        print("\n")

class
First     False
Second     True
Third      True
dtype: bool


     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
17   NaN    male  Second  13.0000         1
20  35.0    male  Second  26.0000         0
21  34.0    male  Second  13.0000         1


    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1
4  35.0    male  Third   8.0500         0
5   NaN    male  Third   8.4583         0
7   2.0    male  Third  21.0750         0




### 6. 멀티 인덱스
- 판다스 : 행 인덱스를 여러 레벨로 구현할 수 있도록 멀티 인덱스 클래스 지원

In [77]:
# class, sex 열 기준으로 분할
grouped = df.groupby(['class','sex'])

gdf = grouped.mean()
print(gdf)
print("\n")
print(type(gdf))

# class가 First인 행 선택해 출력
print(gdf.loc['First'])

                     age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447


<class 'pandas.core.frame.DataFrame'>
              age        fare  survived
sex                                    
female  34.611765  106.125798  0.968085
male    41.281386   67.226127  0.368852


In [78]:
# class가 First, sex 값이 female인 행 선택해 출력
print(gdf.loc[('First','female')])

age          34.611765
fare        106.125798
survived      0.968085
Name: (First, female), dtype: float64


In [79]:
# xs 인덱서 사용
print(gdf.xs('male',level='sex'))
# 나이가 많을수록 상대적으로 비싼 요금 지불하고 높은 등급의 객실 이용하는 경향이
# 있음을 확인할 수 있다.

              age       fare  survived
class                                 
First   41.281386  67.226127  0.368852
Second  30.740707  19.741782  0.157407
Third   26.507589  12.661633  0.135447


### 7. 피벗
- pivot_table()
    - 구성요소 : 행 인덱스, 열 인덱스, 데이터 값, 데이터 집계 함수

In [81]:
# 클래스, 성별 별 평균 나이
pdf1 = pd.pivot_table(df, index='class', columns='sex', values='age', aggfunc='mean')
pdf1.head()

sex,female,male
class,,
First,34.611765,41.281386
Second,28.722973,30.740707
Third,21.750000,26.507589


In [82]:
# 값에 적용하는 집계 함수 2개 이상 지정 가능
pdf2 = pd.pivot_table(df, index='class',
                     columns='sex', values='survived',
                     aggfunc = ['mean','sum'])
pdf2.head()

mean              sum     
sex       female      male female male
class                                 
First   0.968085  0.368852     91   45
Second  0.921053  0.157407     70   17
Third   0.500000  0.135447     72   47

In [83]:
# 행, 열, 값에 사용할 열 2개 이상 지정 가능
pdf3 = pd.pivot_table(df, index=['class','sex'],
                     columns='survived', values=['age','fare'],
                     aggfunc=['mean','max'])
print(pdf3.head())
print("\n")
# 행, 열 구조 살펴보기
print(pdf3.index)
print(pdf3.columns)

                    mean                                      max        \
                     age                   fare               age         
survived               0          1           0           1     0     1   
class  sex                                                                
First  female  25.666667  34.939024  110.604167  105.978159  50.0  63.0   
       male    44.581967  36.248000   62.894910   74.637320  71.0  80.0   
Second female  36.000000  28.080882   18.250000   22.288989  57.0  55.0   
       male    33.369048  16.022000   19.488965   21.095100  70.0  62.0   
Third  female  23.818182  19.329787   19.773093   12.464526  48.0  63.0   

                                 
                 fare            
survived            0         1  
class  sex                       
First  female  151.55  512.3292  
       male    263.00  512.3292  
Second female   26.00   65.0000  
       male     73.50   39.0000  
Third  female   69.55   31.3875  


MultiIndex([( 'F

In [84]:
# xs 인덱서 : 기본값으로 행 인덱서에 접근, default : axis = 0
print(pdf3.xs('First'))

               mean                                      max                \
                age                   fare               age          fare   
survived          0          1           0           1     0     1       0   
sex                                                                          
female    25.666667  34.939024  110.604167  105.978159  50.0  63.0  151.55   
male      44.581967  36.248000   62.894910   74.637320  71.0  80.0  263.00   

                    
                    
survived         1  
sex                 
female    512.3292  
male      512.3292  


In [86]:
print(pdf3.xs(('First','female'))) # 튜플로 전달

            survived
mean  age   0            25.666667
            1            34.939024
      fare  0           110.604167
            1           105.978159
max   age   0            50.000000
            1            63.000000
      fare  0           151.550000
            1           512.329200
Name: (First, female), dtype: float64


In [87]:
print(pdf3.xs('male',level='sex')) # 행 인덱스 레벨 직접 지정

               mean                                    max                \
                age                  fare              age          fare   
survived          0          1          0          1     0     1       0   
class                                                                      
First     44.581967  36.248000  62.894910  74.637320  71.0  80.0  263.00   
Second    33.369048  16.022000  19.488965  21.095100  70.0  62.0   73.50   
Third     27.255814  22.274211  12.204469  15.579696  74.0  45.0   69.55   

                    
                    
survived         1  
class               
First     512.3292  
Second     39.0000  
Third      56.4958  


In [88]:
print(pdf3.xs(('Second','male'), level=[0,'sex']))

                  mean                               max                  
                   age               fare            age        fare      
survived             0       1          0        1     0     1     0     1
class  sex                                                                
Second male  33.369048  16.022  19.488965  21.0951  70.0  62.0  73.5  39.0


In [89]:
# 열 인덱스 접근
print(pdf3.xs('mean', axis=1))

                     age                   fare            
survived               0          1           0           1
class  sex                                                 
First  female  25.666667  34.939024  110.604167  105.978159
       male    44.581967  36.248000   62.894910   74.637320
Second female  36.000000  28.080882   18.250000   22.288989
       male    33.369048  16.022000   19.488965   21.095100
Third  female  23.818182  19.329787   19.773093   12.464526
       male    27.255814  22.274211   12.204469   15.579696


In [90]:
print(pdf3.xs(('mean','age'),axis=1))

survived               0          1
class  sex                         
First  female  25.666667  34.939024
       male    44.581967  36.248000
Second female  36.000000  28.080882
       male    33.369048  16.022000
Third  female  23.818182  19.329787
       male    27.255814  22.274211


In [91]:
# 열 인덱스 레벨 직접 지정
print(pdf3.xs(1,level='survived', axis=1))

                    mean               max          
                     age        fare   age      fare
class  sex                                          
First  female  34.939024  105.978159  63.0  512.3292
       male    36.248000   74.637320  80.0  512.3292
Second female  28.080882   22.288989  55.0   65.0000
       male    16.022000   21.095100  62.0   39.0000
Third  female  19.329787   12.464526  63.0   31.3875
       male    22.274211   15.579696  45.0   56.4958


In [92]:
# 구조받지 못한 승객들 객실 요금의 최대값
print(pdf3.xs(('max','fare',0), level=[0,1,2],axis=1))

                  max
                 fare
survived            0
class  sex           
First  female  151.55
       male    263.00
Second female   26.00
       male     73.50
Third  female   69.55
       male     69.55
